### Проект 1. Исследование поведения пользователей
#### Игра Quiz Freeze

##### Задачи проекта - проверить следующие гипотезы:
###### 1. Есть ли зависимость между выбранным уровнем сложности и вероятностью оплаты.
###### 2. Различается ли временной промежуток между регистрацией и оплатой у групп пользователей с разным уровнем сложности.

In [12]:
import pandas as pd